# Cleaning Data

In [57]:
import time
#from selenium import webdriver
from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import re
import os
from lxml import etree
import lxml.html, lxml.html.clean
from urllib.parse import urlparse
from urllib.parse import parse_qs
import pandas as pd
from itertools import product
import time
import regex
import numpy as np

#driver = webdriver.Chrome(ChromeDriverManager().install())
verbose_flag = True
# htmlSource = driver.page_source

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

### Load data


In [58]:
path_data_scientist = r"C:\Users\RSYeo\Projects\data\data_scientist_2022_03_03.ftr"
path_data_engineer =r"C:\Users\RSYeo\Projects\data\data_engineer_2022_03_03.ftr"
path_data_analyst = r"C:\Users\RSYeo\Projects\data\data_analyst_2022_03_03.ftr"


files = [path_data_scientist, path_data_engineer, path_data_analyst]

In [59]:
df_data_careers = None

start = True
for file_path in files:
    df_read = pd.read_feather(file_path)

    if(start):
        df_data_careers = df_read.copy()
        start = False

    else:
        df_data_careers = pd.concat([df_data_careers, df_read])

df_data_careers.reset_index(drop=True, inplace=True)
        


### Start cleaning data

#### Format ID

In [60]:
# Format job id to be numerical
df_data_careers['job_id'] = df_data_careers.job_id.astype('str').str.extract("(\d+)")

In [61]:
df_data_careers.job_id.nunique()

7851

In [62]:
df_data_careers['job_title'] = df_data_careers['job_title'].str.lower()

#### Drop non-unique jobs

In [63]:
df_data_careers.drop_duplicates(keep='first', subset=['job_id'], inplace=True)

#### Format salary range

In [64]:
#replace wrong values with nan
df_data_careers.loc[df_data_careers['salary_range'].str.contains('Posted'), 'salary_range'] = np.nan

In [65]:
#clean MYR
df_malaysian_salary = df_data_careers.loc[~df_data_careers['salary_range'].isnull() & df_data_careers['salary_range'].str.contains('MYR'), :]


In [66]:
df_malaysian_salary.head()

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state
2,4832121,sql database administrator (数据库管理员),"MYR 15,000 - MYR 25,000","Job DescriptionJob Requirements: Must speak, r...",Senior Executive,3 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Network/Sy...",Morivy Data and Technology Inc.,None,1 - 50 Employees,Computer / Information Technology (Software),/en/job/sql-database-administrator-%E6%95%B0%E...,data scientist,negeri-sembilan
3,4839272,software engineer,"MYR 3,000 - MYR 4,200",Job Description QualificationDegree or higher ...,Junior Executive,2 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",TAIACE ENERGY SDN BHD,1006041-X,1 - 50 Employees,Environment/Health/Safety,/en/job/software-engineer-4839272?jobId=jobstr...,data scientist,negeri-sembilan
4,4852008,manufacturing engineer- fresh graduates (1 yea...,"MYR 3,000 - MYR 3,500",Job HighlightsCareer AdvancementDaily Meal All...,Entry Level,None,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Engineering, Electrical, Mechanical",Plexus Manufacturing Sdn Bhd,399136-M,More than 5000 Employees,Electrical & Electronics,/en/job/manufacturing-engineer-fresh-graduates...,data scientist,negeri-sembilan
7,4819995,system support executive,"MYR 1,500 - MYR 2,000",Job DescriptionSystem Support ExecutiveDeliver...,Junior Executive,2 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",Infomax Supplies,001423432-A,1 - 50 Employees,Computer/Information Technology (Hardware),/en/job/system-support-executive-4819995?jobId...,data scientist,negeri-sembilan
8,4831057,junior chemist product development laboratory,"MYR 2,500 - MYR 3,300",Job Highlights5-day weekProgressive working en...,Junior Executive,2 years,Full-Time,"Professional Certificate, Diploma, Advanced/Hi...","Sciences, Chemistry",MAPEI MALAYSIA SDN BHD,231780-K,51 - 200 Employees,Manufacturing / Production,/en/job/junior-chemist-product-development-lab...,data scientist,negeri-sembilan


In [67]:
extracted_salary = df_malaysian_salary['salary_range'].str.findall("(\d+,\d+)")

In [68]:
df_malaysian_salary.loc[:,'start_salary_range'] = extracted_salary.str[0]
df_malaysian_salary.loc[:,'end_salary_range'] = extracted_salary.str[1]

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\4207735271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary.loc[:,'start_salary_range'] = extracted_salary.str[0]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\4207735271.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary.loc[:,'end_salary_range'] = extracted_salary.str[1]


In [69]:
df_malaysian_salary['start_salary_range'] = df_malaysian_salary['start_salary_range'].str.replace(',','')
df_malaysian_salary.loc[:,'end_salary_range'] = df_malaysian_salary.loc[:,'end_salary_range'].str.replace(',','')

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\2981404410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary['start_salary_range'] = df_malaysian_salary['start_salary_range'].str.replace(',','')
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\2981404410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary.loc[:,'end_salary_range'] = df_malaysian_salary.loc[:,'end_salary_range'].str.replace(',','')


In [70]:
df_malaysian_salary['start_salary_range'] = df_malaysian_salary['start_salary_range'].astype(float)
df_malaysian_salary['end_salary_range'] = df_malaysian_salary['end_salary_range'].astype(float)

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\445547010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary['start_salary_range'] = df_malaysian_salary['start_salary_range'].astype(float)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\445547010.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_malaysian_salary['end_salary_range'] = df_malaysian_salary['end_salary_range'].astype(float)


In [71]:
df_malaysian_salary.shape

(1565, 18)

In [72]:
currencies = df_data_careers['salary_range'].str[:3]

In [73]:
currencies.value_counts()

MYR    1565
SGD       3
HKD       1
Name: salary_range, dtype: int64

In [74]:
#clean SGD
df_singaporean_salary = df_data_careers.loc[~df_data_careers['salary_range'].isnull() & df_data_careers['salary_range'].str.contains('SGD'), :]
extracted_salary_SGD = df_singaporean_salary['salary_range'].str.findall("(\d+,\d+)")
df_singaporean_salary.loc[:,'start_salary_range'] = extracted_salary_SGD.str[0]
df_singaporean_salary.loc[:,'end_salary_range'] = extracted_salary_SGD.str[1]
df_singaporean_salary['start_salary_range'] = df_singaporean_salary['start_salary_range'].str.replace(',','')
df_singaporean_salary.loc[:,'end_salary_range'] = df_singaporean_salary.loc[:,'end_salary_range'].str.replace(',','')

#conversion to MYR
MYR = 3.08
df_singaporean_salary['start_salary_range'] = df_singaporean_salary['start_salary_range'].astype(float)*3.08
df_singaporean_salary['end_salary_range'] = df_singaporean_salary['end_salary_range'].astype(float)*3.08

df_singaporean_salary

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\2162059839.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_singaporean_salary.loc[:,'start_salary_range'] = extracted_salary_SGD.str[0]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\2162059839.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_singaporean_salary.loc[:,'end_salary_range'] = extracted_salary_SGD.str[1]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\2162059839.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state,start_salary_range,end_salary_range
0,4839088,big data lead (work in singapore) - 44565,"SGD 7,000 - SGD 10,500",Job HighlightsExciting future with breakthroug...,Manager,7 years,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",Keysight Technologies Malaysia Sdn. Bhd.,463532-M,2001 - 5000 Employees,Electrical & Electronics,/en/job/big-data-lead-work-in-singapore-44565-...,data scientist,negeri-sembilan,21560.0,32340.0
6595,9299289,"senior commodity specialist (based in penang, ...","SGD 2,000 - SGD 3,000",Job DescriptionResponsibilities:Manages the ou...,Junior Executive,None,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Manufacturing, Purchasing/Material Mgmt",ASM Front-End Manufacturing Singapore Pte Ltd,200310868G,201 - 500 Employees,Semiconductor/Wafer Fabrication,/en/job/senior-commodity-specialist-based-in-p...,data analyst,penang,6160.0,9240.0
6713,9299313,"senior buyer (penang, malaysia)","SGD 2,000 - SGD 3,000",Job DescriptionResponsibilities:Responsible fo...,Junior Executive,None,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Manufacturing, Purchasing/Material Mgmt",ASM Front-End Manufacturing Singapore Pte Ltd,200310868G,201 - 500 Employees,Semiconductor/Wafer Fabrication,/en/job/senior-buyer-penang-malaysia-9299313/o...,data analyst,penang,6160.0,9240.0


In [75]:
#clean HKD
df_hongkong_salary = df_data_careers.loc[~df_data_careers['salary_range'].isnull() & df_data_careers['salary_range'].str.contains('HKD'), :]
extracted_salary_HKD = df_hongkong_salary['salary_range'].str.findall("(\d+,\d+)")
df_hongkong_salary.loc[:,'start_salary_range'] = extracted_salary_HKD.str[0]
df_hongkong_salary.loc[:,'end_salary_range'] = extracted_salary_HKD.str[1]
df_hongkong_salary['start_salary_range'] = df_hongkong_salary['start_salary_range'].str.replace(',','')
df_hongkong_salary.loc[:,'end_salary_range'] = df_hongkong_salary.loc[:,'end_salary_range'].str.replace(',','')

#conversion to MYR
MYR = 0.54
df_hongkong_salary['start_salary_range'] = df_hongkong_salary['start_salary_range'].astype(float)*MYR
df_hongkong_salary['end_salary_range'] = df_hongkong_salary['end_salary_range'].astype(float)*MYR

df_hongkong_salary

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\3422634028.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hongkong_salary.loc[:,'start_salary_range'] = extracted_salary_HKD.str[0]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\3422634028.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hongkong_salary.loc[:,'end_salary_range'] = extracted_salary_HKD.str[1]
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_12800\3422634028.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,job_id,job_title,salary_range,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state,start_salary_range,end_salary_range
3908,4845280,system analyst / senior analyst programmer (ba...,"HKD 25,000 - HKD 35,000",Job DescriptionAbout this roleRegional support...,Senior Executive,5 years,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",Nexperia Malaysia Sdn.Bhd.,244835-W,2001 - 5000 Employees,Semiconductor/Wafer Fabrication,/en/job/system-analyst-senior-analyst-programm...,data analyst,selangor,13500.0,18900.0


In [76]:
df_data_careers['start_salary_range'] = np.nan
df_data_careers['end_salary_range'] = np.nan

In [77]:
def update_data(main_df:pd.DataFrame, new_df:pd.DataFrame, column_names:list):
    for column_name in column_names:
        main_df.loc[new_df.index, column_name] = new_df[column_name]
    
    return main_df

In [78]:
new_data_dfs = [df_malaysian_salary, df_singaporean_salary, df_hongkong_salary]

for df in new_data_dfs:
    df_data_careers = update_data(df_data_careers, df, ['start_salary_range', 'end_salary_range'])


In [79]:
df_data_careers.drop('salary_range', axis=1, inplace=True)

#### Clean experience

In [87]:
df_clean_experience = df_data_careers.copy()
df_clean_experience['experience'] = df_data_careers['experience'].str.replace('years', '')
df_clean_experience['experience'] = df_clean_experience['experience'].str.replace('year', '')

df_clean_experience['experience'] = df_clean_experience['experience'].str.strip()
df_clean_experience.loc[df_clean_experience[df_clean_experience['experience'].isnull()].index, 'experience'] = '0'
df_clean_experience['experience'] = df_clean_experience['experience'].astype(int)
df_clean_experience.head(10)

df_data_careers = df_clean_experience.copy()

#### Filter relevant jobs

In [126]:
df_data_careers.shape

(7851, 17)

Get all jobs mentioning data

In [103]:
df_data_only = df_data_careers[df_data_careers.job_title.str.contains('data')].copy()

In [104]:
df_no_data = df_data_careers.drop(df_data_only.index)

Get all jobs mentioning computer science

In [120]:
df_computer_science = df_no_data[df_no_data['job_title'].str.contains('science|scientist') & df_no_data['job_title'].str.contains('computer')].copy()

In [121]:
df_no_computer_science = df_no_data.drop(df_computer_science.index)

Get all jobs mentioning machine learning

In [122]:
df_machine_learning = df_no_computer_science[df_no_computer_science['job_title'].str.contains('machine learning')].copy()

In [124]:
df_no_machine_learning = df_no_computer_science.drop(df_machine_learning.index)

Get all software jobs mentioning analytics, machine learning, artificial intelligence in description

In [159]:
df_software_analytics = df_no_machine_learning[df_no_machine_learning['job_title'].str.contains('software') & df_no_machine_learning['job_description'].str.contains('artificial|intelligence|analytics|machine learning')]

In [160]:
df_no_software_analytics = df_no_machine_learning.drop(df_software_analytics.index)

Get all jobs mentioning artificial intelligence and machine learning

In [165]:
df_misc = df_no_software_analytics[df_no_software_analytics['job_description'].str.contains('machine learning|artificial intelligence|data science|data engineering')]
df_ai = df_misc[df_misc['job_title'].str.contains('ai')]
df_ai

,job_id,job_title,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state,start_salary_range,end_salary_range
125,4807612,java technical (blockchain & ai),Job DescriptionWe are looking for someone who ...,Junior Executive,1,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",PROTECH DIGITAL SDN. BHD.,954217-T,51 - 200 Employees,Computer / Information Technology (Software),/en/job/java-technical-blockchain-ai-4807612?j...,data scientist,selangor,3000.0,5000.0
138,4824493,artificial intelligence (ai) technologist,Job HighlightsTraining fully funded by Malaysi...,Entry Level,0,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",Selangor Human Resource Development Centre (SH...,5192/96(SEL),1 - 50 Employees,Education,/en/job/artificial-intelligence-ai-technologis...,data scientist,selangor,NaN,NaN
240,4828951,it maintenance officer,Job HighlightsAble to start immediatelyGood ca...,Junior Executive,1,Full-Time,"Professional Certificate, Diploma, Advanced/Hi...","Computer/Information Technology, IT-Network/Sy...",Wonway Manufacturing Sdn Bhd,164070-D,51 - 200 Employees,Manufacturing / Production,/en/job/it-maintenance-officer-4828951?jobId=j...,data scientist,selangor,2000.0,2500.0
471,4817947,campaign manager - (j104884),Job DescriptionYou are: (personality / compete...,Senior Executive,0,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Sales/Marketing, Marketing/Business Dev",Digi Telecommunications Sdn. Bhd.,201283-M,1001 - 2000 Employees,Telecommunication,/en/job/campaign-manager-j104884-4817947?jobId...,data scientist,selangor,NaN,NaN
497,1030723178,sr ai and systems engineer,Job DescriptionDescription- Lead the developme...,Not Specified,0,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",Tapway,1021460-H,1 - 50 Employees,Manufacturing / Production,/en/job/sr-ai-and-systems-engineer-1030723178?...,data scientist,selangor,NaN,NaN
498,1030723137,ai engineer,Job DescriptionDescription- Lead the developme...,Not Specified,0,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",Tapway,1021460-H,1 - 50 Employees,Manufacturing / Production,/en/job/ai-engineer-1030723137?jobId=jobstreet...,data scientist,selangor,NaN,NaN
533,1030522063,ai intern,Job DescriptionAI InternTenure: Internship Pro...,Not Specified,0,Internship,Not Specified,"Computer/Information Technology, IT-Software",DoctorOnCall,561740-T,1 - 50 Employees,Healthcare / Medical,/en/job/ai-intern-1030522063?jobId=jobstreet-m...,data scientist,selangor,NaN,NaN
1046,4836702,ai engineer,Job HighlightsCareer advancementComprehensive ...,Junior Executive,2,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Engineering, Electrical",Mi Equipment (M) Sdn. Bhd.,1005839-H,201 - 500 Employees,Heavy Industrial / Machinery / Equipment,/en/job/ai-engineer-4836702?jobId=jobstreet-my...,data scientist,penang,NaN,NaN
1170,4837894,rma engineer / repair engineer,Job DescriptionJob descriptionJob RequirementC...,Senior Executive,2,Full-Time,Not Specified,"Engineering, Mechanical",Advantech Co. Malaysia Sdn Bhd,719042-T,1 - 50 Employees,Computer/Information Technology (Hardware),/en/job/rma-engineer-repair-engineer-4837894?j...,data scientist,penang,3000.0,5000.0
1228,4816702,ic design engineer (graduate trainee),Job DescriptionKICKSTART YOUR CAREER IN IC DES...,Entry Level,0,Contract,Not Specified,"Engineering, Electronics",DREAM CATCHER CONSULTING SDN. BHD. (PENANG),663670-T,1 - 50 Employees,Consulting (Business & Management),/en/job/ic-design-engineer-graduate-trainee-48...,data scientist,penang,NaN,NaN


In [156]:
df_misc


,job_id,job_title,job_description,job_level,experience,job_type,qualifications,job_specialization,company_name,company_registration,company_size,company_industry,job_link,search_term,state,start_salary_range,end_salary_range
1,4831309,korean content specialist (pharmaceutical) bas...,Job HighlightsBest Workplace EverCompetitive B...,Senior Executive,1,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Others, Publishing",Clarivate (Malaysia) Sdn Bhd,1360469-D,2001 - 5000 Employees,"Consulting (IT, Science, Engineering & Technical)",/en/job/korean-content-specialist-pharmaceutic...,data scientist,negeri-sembilan,NaN,NaN
6,4824547,"frontend developer, reactjs /vuejs",Job DescriptionOur client is one of the world'...,Junior Executive,3,Full-Time,"Diploma, Advanced/Higher/Graduate Diploma","Computer/Information Technology, IT-Software",Michael Page International (Malaysia) Sdn Bhd,914741-W,51 - 200 Employees,Human Resources Management / Consulting,/en/job/frontend-developer-reactjs-vuejs-48245...,data scientist,negeri-sembilan,NaN,NaN
23,4801152,power bi developer,Job Description· OVERALL SCOPE OF WORKTh...,Senior Executive,3,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",Maica Laminates Sdn Bhd,107254-D,201 - 500 Employees,Manufacturing / Production,/en/job/power-bi-developer-4801152?jobId=jobst...,data scientist,kedah,NaN,NaN
27,4850148,senior staff specialist wip flow management,Job DescriptionPart of your life. Part of tomo...,Senior Executive,3,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Engineering, Industrial Engineering",Infineon Technologies (Kulim) Sdn Bhd,679693W,2001 - 5000 Employees,Semiconductor/Wafer Fabrication,/en/job/senior-staff-specialist-wip-flow-manag...,data scientist,kedah,NaN,NaN
39,1030539972,senior costing analyst,Job DescriptionCompany DescriptionABOUT AVERY ...,Not Specified,0,Full-Time,Not Specified,"Computer/Information Technology, IT-Software",Avery Dennison,Avery Dennison,201 - 500 Employees,Manufacturing / Production,/en/job/senior-costing-analyst-1030539972?jobI...,data scientist,kedah,NaN,NaN
42,1030645608,test process engineering intern,Job DescriptionTest Process Engineering Intern...,Not Specified,0,Internship,Not Specified,"Engineering, Electrical",Intel,36420-H,More than 5000 Employees,Semiconductor/Wafer Fabrication,/en/job/test-process-engineering-intern-103064...,data scientist,kedah,NaN,NaN
43,1030601460,process engineer,Job DescriptionCompany DescriptionABOUT AVERY ...,Not Specified,0,Full-Time,Not Specified,"Engineering, Electrical",Avery Dennison,Avery Dennison,201 - 500 Employees,Manufacturing / Production,/en/job/process-engineer-1030601460?jobId=jobs...,data scientist,kedah,NaN,NaN
44,1030540127,procurement senior executive,Job DescriptionCompany DescriptionABOUT AVERY ...,Not Specified,0,Full-Time,Not Specified,"Manufacturing, Purchasing/Material Mgmt",Avery Dennison,Avery Dennison,201 - 500 Employees,Manufacturing / Production,/en/job/procurement-senior-executive-103054012...,data scientist,kedah,NaN,NaN
54,1030730908,inventory control asst. manager,Job DescriptionAt Jabil we strive to make ANYT...,Not Specified,0,Full-Time,Not Specified,"Services, Logistics/Supply Chain",Jabil,839928-K,501 - 1000 Employees,Electrical & Electronics,/en/job/inventory-control-asst-manager-1030730...,data scientist,kedah,NaN,NaN
110,4854724,artificial intelligence software engineer,Job DescriptionJob DescriptionEmerico is at th...,Senior Executive,3,Full-Time,"Bachelor's Degree, Post Graduate Diploma, Prof...","Computer/Information Technology, IT-Software",Emerico Sdn Bhd,1007269-W,51 - 200 Employees,Computer / Information Technology (Software),/en/job/artificial-intelligence-software-engin...,data scientist,selangor,6000.0,12000.0


In [141]:
df_software[df_software['job_description'].str.contains('data')].iloc[0].job_description

'Job Description\xa0QualificationDegree or higher in information technology / computer science or related discipline. Fresh are encouraged to apply.Exposure in network administration, technical support, hardware and software support, and troubleshooting.Must possess good communications skills, analytical and problem solving skills.Good command of both spoken and written in Bahasa and English.Able to work independently as well as a good team player.Good interpersonal skills and able to work as a team.Good sense of urgency and able to work under pressure.Able to identify and demonstrate clear comprehension of customers and organizational needs.Punctuality and confident in client facing ability.Experience with LAMP, WAMP LARAGON, APACHE, XAMPKnowledgeable with PHP / HTML / JAVASCRIPT / JQUERY / AJAX / CSS / JS/ BOOTSTRAP 4 / BOOTSTRAPRequired Database Skill PGSQL / NOSQL / MONGODB / MYSQL / MSSQLExperience in Network, Serve and Data Communication, Cloud Management System (eg. Synology NAS

In [ ]:
df_data_careers[~df_data_careers['job_title'].str.contains('data')]

In [100]:
df_data_careers[df_data_careers['job_description'].str.contains('big data')].iloc[0]['job_title']

'big data lead (work in singapore) - 44565'